In [ ]:
import dotenv
import os
import json
from textwrap import dedent
import gradio as gr

from openai import OpenAI
from swarm import Swarm, Agent

dotenv.load_dotenv(".env", override=True)
openai_client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
client = Swarm(client=openai_client)




## Agent handoff to another Agent 

In [ ]:

english_agent = Agent(
    name="English Agent",
    instructions="You only speak English.",
)

arabic_agent = Agent(
    name="Arabic Agent",
    instructions="You only speak Arabic.",
)


def transfer_to_arabic_agent():
    """Transfer arabic speaking users immediately."""
    return arabic_agent


english_agent.functions.append(transfer_to_arabic_agent)

messages = [{"role": "user", "content": "السلام عليكم، كيف حالك؟" }]
response = client.run(agent=english_agent, messages=messages)
messages.extend(response.messages)
print(*[ f"{m['role']} : {m['content']}" for m in messages], sep="\n")

# Agent Tools usage

In [ ]:

def get_weather(location, time="now"):
    """Get the current weather in a given location. Location MUST be a city."""
    print(f"Getting weather for {location} at {time} from wather.com!!")
    print("Done!\n")
    return json.dumps({"location": location, "temperature": "65", "time": time})


def send_email(recipient, subject, body):
    print("Sending email...")
    print(f"To: {recipient}")
    print(f"Subject: {subject}")
    print(f"Body: {body}")
    print(f"Done!\n")
    return "Sent!"


tool_agent = Agent(
    name="Tool Agent",
    instructions="You are a helpful agent.",
    functions=[get_weather, send_email],
)

messages = []
messages.extend([{"role": "user", "content": "ايه احوال الجو اليوم؟"}])
response = client.run(agent=tool_agent, messages=messages)
messages.extend(response.messages)

messages.extend([{"role": "user", "content": "القاهرة"}])
response = client.run(agent=tool_agent, messages=messages)
messages.extend(response.messages)

messages.extend([{"role": "user", "content": dedent("""
اريد ارسال بريد الكتروني الى احمد فى code-quests.com
بعنوان: اجتماع الغد سيكون رائعا باذن الله
والنص: سنتعلم الكثير ونستمتع بالوقت معا
""")
}])
response = client.run(agent=tool_agent, messages=messages)
messages.extend(response.messages)

print(*[ f"{m['role']} : {m['content']}" for m in messages], sep="\n\n")

# Agent Routing and Selection

In [4]:

def process_refund(item_id, reason="NOT SPECIFIED"):
    """Refund an item. Refund an item. Make sure you have the item_id of the form item_... Ask for user confirmation before processing the refund."""
    print(f"[mock] Refunding item {item_id} because {reason}...")
    return "Refund Success!"


def apply_discount():
    """Apply a discount to the user's cart."""
    print("[mock] Applying discount...")
    return "Applied discount of 11%"


triage_agent = Agent(
    name="Triage Agent",
    instructions="Determine which agent is best suited to handle the user's request, and transfer the conversation to that agent.",
)
sales_agent = Agent(
    name="Sales Agent",
    instructions="Be super enthusiastic about selling bees.",
)
refunds_agent = Agent(
    name="Refunds Agent",
    instructions="Help the user with a refund. If the reason is that it was too expensive, offer the user a discount code. If they insist, then process the refund.",
    functions=[process_refund, apply_discount],
)


def transfer_back_to_triage():
    """Call this function if a user is asking about a topic that is not handled by the current agent."""
    return triage_agent


def transfer_to_sales():
    return sales_agent


def transfer_to_refunds():
    return refunds_agent


triage_agent.functions = [transfer_to_sales, transfer_to_refunds]
sales_agent.functions.append(transfer_back_to_triage)
refunds_agent.functions.append(transfer_back_to_triage)




In [ ]:
messages = [{"role": "user", "content": "بدى اشترى نحلة"}]
response = client.run(agent=triage_agent, messages=messages)
messages.extend(response.messages)
print(*[ f"{m['role']} : {m['content']}" for m in messages], sep="\n")


In [ ]:
messages = [{"role": "user", "content": "أنا عايز ارجع النحلة دى"}]
response = client.run(agent=triage_agent, messages=messages)
messages.extend(response.messages)

# user insists on refund 
messages.extend([{"role": "user", "content": "أصل النحل بتاعكم غالى"}])
response = client.run(agent=response.agent, messages=messages)
messages.extend(response.messages)


# user insists on refund 
messages.extend([{"role": "user", "content": "مش عايز خصم، أنا عايز فلوسى"}])
response = client.run(agent=response.agent, messages=messages)
messages.extend(response.messages)

# user insists on refund 
messages.extend([{"role": "user", "content": "النحلة item_100"}])
response = client.run(agent=response.agent, messages=messages)
messages.extend(response.messages)


print(*[ f"{m['role']} : {m['content']}" for m in messages], sep="\n\n")



# 

In [ ]:
# Make it Interactive with Gradio!

In [ ]:
def chat_with_agent(message, history):
    messages = [{"role": "user", "content": message}]
    response = client.run(agent=triage_agent, messages=messages)
    messages.extend(response.messages)
    
    return messages[-1]["content"]

demo = gr.ChatInterface(fn=chat_with_agent,
        type="messages")

demo.launch()